# CELL 0 - run only when emnist or svhn datasest are needed

In [1]:
!pip install kaggle

In [2]:
# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
!mkdir /.kaggle
!mv kaggle.json /.kaggle
!mv /.kaggle /root/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download andrewmvd/ocular-disease-recognition-odir5k

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k
License(s): other
100% 1.62G/1.62G [00:17<00:00, 33.7MB/s]
100% 1.62G/1.62G [00:17<00:00, 99.5MB/s]


In [4]:
!unzip ocular-disease-recognition-odir5k.zip

Datele de ieșire de afișat au fost trunchiate la ultimele 5000 linii.
  inflating: preprocessed_images/2179_left.jpg  
  inflating: preprocessed_images/2179_right.jpg  
  inflating: preprocessed_images/217_left.jpg  
  inflating: preprocessed_images/217_right.jpg  
  inflating: preprocessed_images/2180_left.jpg  
  inflating: preprocessed_images/2180_right.jpg  
  inflating: preprocessed_images/2181_left.jpg  
  inflating: preprocessed_images/2181_right.jpg  
  inflating: preprocessed_images/2182_left.jpg  
  inflating: preprocessed_images/2182_right.jpg  
  inflating: preprocessed_images/2183_left.jpg  
  inflating: preprocessed_images/2183_right.jpg  
  inflating: preprocessed_images/2184_left.jpg  
  inflating: preprocessed_images/2184_right.jpg  
  inflating: preprocessed_images/2185_left.jpg  
  inflating: preprocessed_images/2185_right.jpg  
  inflating: preprocessed_images/2187_left.jpg  
  inflating: preprocessed_images/2187_right.jpg  
  inflating: preprocessed_images/2189_lef

In [5]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
imsize=64;
datagen=False
dataset='ocular'

# poreluare etichetare in format df
import pandas as pd
import numpy as np
df = pd.read_csv("/content/full_df.csv")
df.shape[0]

#etichetare clase
def has_normal(text):
    if "normal" in text:
        return 1
    else:
        return 0
def has_cataract(text):
    if "cataract" in text:
        return 1
    else:
        return 0
def has_diabetes(text):
    if "retinopathy" in text:
        return 1
    else:
        return 0
def has_glaucoma(text):
    if "glaucoma" in text:
        return 1
    else:
        return 0
def has_degeneration(text):
    if "age-related macular degeneration" in text:
        return 1
    else:
        return 0
def has_hypertension(text):
    if "hypertensive" in text:
        return 1
    else:
        return 0
def has_myopia(text):
    if "pathological myopia" in text:
        return 1
    else:
        return 0

df["left_normal"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_normal(x))
df["right_normal"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_normal(x))
left_normal = df.loc[(df.N ==1) & (df.left_normal == 1)]["Left-Fundus"].values
left_normal
right_normal = df.loc[(df.N ==1) & (df.right_normal == 1)]["Right-Fundus"].values
right_normal
print("Number of images in left normal: {}".format(len(left_normal)))
print("Number of images in right normal: {}".format(len(right_normal)))
print()


df["left_cataract"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
df["right_cataract"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
left_cataract = df.loc[(df.C ==1) & (df.left_cataract == 1)]["Left-Fundus"].values
left_cataract
right_cataract = df.loc[(df.C ==1) & (df.right_cataract == 1)]["Right-Fundus"].values
right_cataract
print("Number of images in left cataract: {}".format(len(left_cataract)))
print("Number of images in right cataract: {}".format(len(right_cataract)))
print()


df["left_diabetes"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_diabetes(x))
df["right_diabetes"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_diabetes(x))
left_diabetes = df.loc[(df.D ==1) & (df.left_diabetes == 1)]["Left-Fundus"].values
right_diabetes = df.loc[(df.D ==1) & (df.right_diabetes == 1)]["Right-Fundus"].values
diabetes_images = np.concatenate((left_diabetes, right_diabetes))
print("Number of images in left diabetes: {}".format(len(left_diabetes)))
print("Number of images in right diabetes: {}".format(len(right_diabetes)))
print()


df["left_glaucoma"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_glaucoma(x))
df["right_glaucoma"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_glaucoma(x))
left_glaucoma = df.loc[(df.G ==1) & (df.left_glaucoma == 1)]["Left-Fundus"].values
left_glaucoma
right_glaucoma = df.loc[(df.G ==1) & (df.right_glaucoma == 1)]["Right-Fundus"].values
right_glaucoma
print("Number of images in left glaucoma: {}".format(len(left_glaucoma)))
print("Number of images in right glaucoma: {}".format(len(right_glaucoma)))
print()


df["left_degeneration"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_degeneration(x))
df["right_degeneration"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_degeneration(x))
left_degeneration = df.loc[(df.A ==1) & (df.left_degeneration == 1)]["Left-Fundus"].values
left_degeneration
right_degeneration = df.loc[(df.A ==1) & (df.right_degeneration == 1)]["Right-Fundus"].values
right_degeneration
print("Number of images in left degeneration: {}".format(len(left_degeneration)))
print("Number of images in right degeneration: {}".format(len(right_degeneration)))
print()


df["left_myopia"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_myopia(x))
df["right_myopia"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_myopia(x))
left_myopia = df.loc[(df.M ==1) & (df.left_myopia == 1)]["Left-Fundus"].values
left_myopia
right_myopia = df.loc[(df.M ==1) & (df.right_myopia == 1)]["Right-Fundus"].values
right_myopia
print("Number of images in left myopia: {}".format(len(left_myopia)))
print("Number of images in right myopia: {}".format(len(right_myopia)))
print()


df["left_hypertension"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_hypertension(x))
df["right_hypertension"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_hypertension(x))
left_hypertension = df.loc[(df.H ==1) & (df.left_hypertension == 1)]["Left-Fundus"].values
left_hypertension
right_hypertension = df.loc[(df.H ==1) & (df.right_hypertension == 1)]["Right-Fundus"].values
right_hypertension
print("Number of images in left hypertension: {}".format(len(left_hypertension)))
print("Number of images in right hypertension: {}".format(len(right_hypertension)))
print()


left_abnormalities = df.loc[(df.C ==0) & (df.N ==0) & (df.D ==0) & (df.G ==0) & (df.A ==0)& (df.M ==0) & (df.H ==0)]["Left-Fundus"].sample(250,random_state=42).values
left_abnormalities
right_abnormalities = df.loc[(df.C ==0) & (df.N ==0) & (df.D ==0) & (df.G ==0) & (df.A ==0)& (df.M ==0) & (df.H ==0) ]["Right-Fundus"].sample(250,random_state=42).values
right_abnormalities
print("Number of images in left abnormalities: {}".format(len(left_abnormalities)))
print("Number of images in right abnormalities: {}".format(len(right_abnormalities)))
print()


cataract = np.concatenate((left_cataract,right_cataract),axis=0)
normal = np.concatenate((left_normal,right_normal),axis=0)
glaucoma = np.concatenate((left_glaucoma,right_glaucoma),axis=0)
diabetes = np.concatenate((left_diabetes,right_diabetes),axis=0)
degeneration = np.concatenate((left_degeneration,right_degeneration),axis=0)
myopia = np.concatenate((left_myopia,right_myopia),axis=0)
hypertension = np.concatenate((left_hypertension,right_hypertension),axis=0)
abnormalities = np.concatenate((left_abnormalities,right_abnormalities),axis=0)

print("Total cataract images: ",len(cataract))
print("Total normal images: ",len(normal))
print("Total galucoma images: ",len(glaucoma))
print("Total diabetes images: ",len(diabetes))
print("Total degeneration images: ",len(degeneration))
print("Total myopia images: ",len(myopia))
print("Total hypertension images: ",len(hypertension))
print("Total abnormalities images: ",len(abnormalities))
print()

def load_odir_data(Array_of_Images):
    Images = []
    Labels = []
    count = -1
    for i in Array_of_Images:
        count += 1
        print(count)
        count1 = 0;
        for j in i:
            count1 += 1
            #empty_Label = np.array([0,0,0,0])
            empty_Label= np.zeros(num_classes)  # pentru un numar arbitrar de clase
            if(j in os.listdir("/content/ODIR-5K/ODIR-5K/Testing Images/")):
                fileName = "/content/ODIR-5K/ODIR-5K/Testing Images/"+j
            elif(j in os.listdir("/content/ODIR-5K/ODIR-5K/Training Images/")):
                fileName = "/content/ODIR-5K/ODIR-5K/Training Images/"+j
            else:
                fileName = "/content/preprocessed_images/"+j
            images = cv2.imread(fileName)
#             print(images.shape)
            images = cv2.resize(images, (imsize,imsize), interpolation = cv2.INTER_NEAREST)
            Images.append(np.array(images))
            empty_Label[count] = 1
            Labels.append(empty_Label)

    return np.array(Images), np.array(Labels)

#--- aici se face o alegere pe maximum de 8 clase (perf. mai proasta)
my_categories=[normal, cataract, glaucoma, myopia]  # 4 clase (se pot organiza oricum - cu 6 clase perf. e mai slaba )
#my_categories=[normal, cataract, glaucoma, degeneration, myopia, hypertension]


num_classes=len(my_categories)
import time as ti
t1=ti.time()
Images, Labels = load_odir_data(my_categories)
t2=ti.time()
print('Durata creare baza de date: ', t2-t1, 'secunde',' Numar clase: ', num_classes)

Number of images in left normal: 2100
Number of images in right normal: 2101

Number of images in left cataract: 304
Number of images in right cataract: 290

Number of images in left diabetes: 1689
Number of images in right diabetes: 1759

Number of images in left glaucoma: 332
Number of images in right glaucoma: 284

Number of images in left degeneration: 266
Number of images in right degeneration: 285

Number of images in left myopia: 216
Number of images in right myopia: 241

Number of images in left hypertension: 191
Number of images in right hypertension: 191

Number of images in left abnormalities: 250
Number of images in right abnormalities: 250

Total cataract images:  594
Total normal images:  4201
Total galucoma images:  616
Total diabetes images:  3448
Total degeneration images:  551
Total myopia images:  457
Total hypertension images:  382
Total abnormalities images:  500

0
1
2
3
Durata creare baza de date:  360.5529978275299 secunde  Numar clase:  4


In [8]:
input_shape=(imsize,imsize,3)
batch_size=16
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, random_state=42)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255; x_test /=255  # scalare [0,1]
print('Numar clase: ',np.shape(y_train)[1])
print('Date intrare: ',input_shape)

NameError: name 'Images' is not defined